In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 

In [4]:
df = pd.read_csv("S06-hw-dataset-04.csv")
df.head()
df.info()
df.describe(include="all")


FileNotFoundError: [Errno 2] No such file or directory: 'S06-hw-dataset-04.csv'

In [1]:
df.isna().sum().sort_values(ascending=False)

# df["target"].value_counts()

NameError: name 'df' is not defined

In [16]:
X = df.drop(columns=["target", "id"], errors = "ignore")
y = df["target"]

X.shape, y.shape

((25000, 60), (25000,))

In [18]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, 
    y,
    test_size = 0.25,
    random_state = 42,
    stratify = y
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18750, 60), (6250, 60), (18750,), (6250,))

мы фиксируем случайность разбиения, чтобы при каждом запуске получались одинаковые train/test. это нужно для правильного сравнения моделей и воспроизводимости результата

стратификация важна, потому что она сохраняет пропорции классво в train и test. без этого можно случайно получить неправильный test, и метрики будут неверными

In [ ]:

def calc_metrics(y_true, y_pred):
    return {
        "accuracy": sklearn.metrics.accuracy_score(y_true, y_pred),
        "precision": sklearn.metrics.precision_score(y_true, y_pred, zero_division = 0), 
        "recall": sklearn.metrics.recall_score(y_true, y_pred, zero_division = 0), 
        "f1": sklearn.metrics.f1_score(y_true, y_pred, zero_division = 0), 
    }

def print_metrics(title, y_true, y_pred):
    mets = calc_metrics(y_true, y_pred)
    print(title)
    for i, j in mets.items():
        print(i, j)



In [ ]:
# DummyClassifier

dummy = sklearn.dummy.DummyClassifier(strategy="most_frequent", random_state=42)
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)
print_metrics("dummy (most_frequent)", y_test, y_pred_dummy)

dummy - это базовая нулевая модель, которая вообще не учится и выбирает самый частый класс. любая модель должна быть лучше этой глупой модели.

In [ ]:
# LogisticRegression

lr_pipe = sklearn.pipeline.Pipeline(
    steps = [
        ("scaler", sklearn.preprocessing.StandardScaler()),
        ("model", sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=42)),
    ]
)

lr_pipe.fit(X_train, y_train)
y_pred_lr = lr_pipe.predict(X_test)

print_metrics("log res + standart scaler", y_test, y_pred_lr)

логистическая регрессия - довольно быстрый и сильный baseline. масштабирование признаков нужно, чтобы модель работала при разных порядков величин у фич.

In [21]:
cv = sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_on_test(name, model):
    y_pred = model.predict(X_test)
    m = calc_metrics(y_test, y_pred)
    return {"model": name, **m}
result = []
best_model = {}

In [23]:
dectree = sklearn.tree.DecisionTreeClassifier(random_state=42)

dectree_grid = {
    "max_depth": [None, 3, 5, 8, 10, 14], 
    "min_samples_leaf": [1, 2, 6, 13, 15]
}

dectree_search = sklearn.model_selection.GridSearchCV(
    estimator = dectree, 
    param_grid = dectree_grid, 
    scoring = "f1", 
    cv = cv, 
    n_jobs =- 1
)

dectree_search.fit(X_train, y_train)
best_dectree = dectree_search.best_estimator_

print("DecTree best params:", dectree_search.best_params_)
best_model["DecTree"] = best_dectree

DecTree best params: {'max_depth': 14, 'min_samples_leaf': 6}


In [25]:
randforest = sklearn.ensemble.RandomForestClassifier(random_state=42)

randforest_grid = {
    "n_estimators": [200, 500], 
    "max_depth": [None, 5, 10, 15], 
    "max_features": ["sqrt", "log2", None],
    "min_samples_leaf": [1, 2, 5, 10], 
}

randforest_search = sklearn.model_selection.GridSearchCV(
    estimator = randforest, 
    param_grid = randforest_grid,
    scoring = "f1",
    cv = cv, 
    n_jobs =- 1,
)

randforest_search.fit(X_train, y_train)
best_randforest = randforest_search.best_estimator_

print("RandForest best params: ", randforest_search.best_params_)
best_model["RandomForest"] = best_randforest

KeyboardInterrupt: 

In [ ]:
ada = sklearn.ensemble.AdaBoostClassifier(random_state = 42)

ada_grid = 